In [2]:
import pandas as pd

import tensorflow as tf

import gym

In [5]:
df_year = pd.read_pickle("data/ETH_USD_2017_2018_15min.pkl")
display(df_year.head(5))

,time,low,high,open,close,volume
0,2017-01-01 00:00:00,8.11,8.16,8.16,8.16,378.420945
1,2017-01-01 00:15:00,8.16,8.24,8.16,8.24,252.619480
2,2017-01-01 00:30:00,8.24,8.27,8.24,8.27,237.637246
3,2017-01-01 00:45:00,8.25,8.27,8.27,8.26,109.079011
4,2017-01-01 01:00:00,8.26,8.29,8.26,8.29,270.898827


In [7]:
n_features = 4 # Low, High Close, Volume
n_days_window = 30
n_window = (n_days_window * 24 * 60) // 15

In [10]:
class A():
    def _get(self):
        return 1

a = A()
b = a.get()

AttributeError: 'A' object has no attribute 'get'

TypeError: _get() takes 0 positional arguments but 1 was given